### Mostly overlapped with `generate_tables.py` but older version

In [95]:
import os
import re
import pandas as pd

#### Get input files

In [96]:
file_path = os.path.join(os.getcwd(), '../../input_data/BioNLP-OST-2019_BB-norm_train') # BioNLP-OST-2019_BB-norm_dev
os.listdir(file_path)
input_file = [_ for _ in os.listdir(file_path) if _.endswith('.a1')]
label_file = [_ for _ in os.listdir(file_path) if _.endswith('.a2')]
# For training and dev data set, every input file comes with a label file
assert (len(input_file)==len(label_file))

#### Output the id, category, name and other information of the entities in training and dev datasets to file
输出训练集和验证集的实体编号，实体类别，实体名称等信息宽表

In [4]:
pattern='.*norm-(.*)\..*'
with open('../../input_data/wide_tables/entity_list_dev.tsv', 'w+') as f_handler:  #entity_list_dev.csv
    header=True
    for f in input_file:
        f_id = re.search(pattern, f).group(1)
        
        # Parse input files, with format of: label \t category \s [start \s end]rep  \t name
        f_in = pd.read_csv(os.path.join(file_path, f), sep='\t', header=None)
        f_in.columns=['entity_id', 'mix', 'name']
        f_in[['category', 'positions']] = f_in.mix.str.split(n=1, expand=True)
        f_in.drop(columns=['mix'], inplace=True)
        f_in.insert(0, 'text_id', f_id)
        if header:
            f_in.to_csv(f_handler, index=False, sep='\t')
            header=False
        else:
            f_in.to_csv(f_handler, index=False, header=False, sep='\t')
f_handler.close()

#### Output the id, category, name and other information of the entities, together with the id, name and other information of the normalized concepts, in training and dev datasets to file

输出训练集和验证集的实体编号，实体类别，实体名称等，与标准化后正确编号，名称等信息宽表

Parse reference dictionaries: OBT and NCBI taxdump

In [59]:
ref_obt = pd.read_csv(os.path.join(os.getcwd(), '../../input_data/OBT.txt'), sep='|')
ref_obt.head()

,id,name
0,3381,fermented cheese
1,2017,enriched dough
2,2100,lean dough
3,3144,fermented cereal-based product
4,2417,crusty bread


In [98]:
ref_obt.columns=['dict_id', 'dict_name']
# For the OBT dict with id of format: xxxxxx(number, no 'OBT:' prefix)
# for i in range(ref_obt.shape[0]):
#     ref_obt.dict_id[i] = 'OBT:'+'0'*(6-len(str(ref_obt.dict_id[i])))+str(ref_obt.dict_id[i])
# ref_obt.dict_id = ref_obt.dict_id.astype(str)

ref_obt.head()

,dict_id,dict_name
0,OBT:003381,fermented cheese
1,OBT:002017,enriched dough
2,OBT:002100,lean dough
3,OBT:003144,fermented cereal-based product
4,OBT:002417,crusty bread


In [69]:
ref_tax_raw = pd.read_csv(os.path.join(os.getcwd(), '../../input_data/TAX1.txt'), sep='|')
ref_tax_raw.head()
# ref_tax_raw = ref_tax_raw.iloc[:50,:]

,tax_id,name_txt,name_class
0,1.0,all,synonym
1,1.0,root,scientific name
2,2.0,Bacteria,scientific name
3,2.0,Monera,in-part
4,2.0,Procaryotae,in-part


In [73]:
ref_tax = ref_tax_raw.groupby('tax_id').agg(lambda x: x.tolist())
ref_tax.reset_index(inplace=True)
ref_tax.columns=['dict_id', 'dict_name', 'dict_name_class']
ref_tax.dict_id = ref_tax.dict_id.astype(int).astype(str)
ref_tax.head()

,dict_id,dict_name,dict_name_class
0,1,"[all, root]","[synonym, scientific name]"
1,2,"[Bacteria, Monera, Procaryotae, Prokaryota, Pr...","[scientific name, in-part, in-part, in-part, i..."
2,6,"[Azorhizobium, Azorhizobium Dreyfus et al. 198...","[scientific name, authority]"
3,7,"[ATCC 43989, Azorhizobium caulinodans, Azorhiz...","[type material, scientific name, authority, eq..."
4,9,"[Acyrthosiphon pisum symbiont P, Buchnera aphi...","[includes, scientific name, authority, includes]"


In [76]:
pattern='.*norm-(.*)\..*'
header=True
for f in input_file:
    f_id = re.search(pattern, f).group(1)
    
    # Parse input files, with format of: label \t category \s [start \s end]rep  \t name
    f_in = pd.read_csv(os.path.join(file_path, f), sep='\t', header=None)
    f_in.columns=['entity_id', 'mix', 'name']
    f_in[['category', 'positions']] = f_in.mix.str.split(n=1, expand=True)
    f_in.drop(columns=['mix'], inplace=True)
    f_in.insert(0, 'text_id', f_id)

    # Parse output files, with format of: label \t category \s entity_lable \s referent
    f_out = pd.read_csv(os.path.join(file_path, f[:-1]+'2'), sep='\t|\s', header=None)
    f_out.columns=['standard_id', 'dict_type', 'entity_id', 'dict_id']
    f_out.entity_id = f_out.entity_id.str.lstrip('Annotation:')
    f_out.dict_id = f_out.dict_id.str.lstrip('Referent:')    
    f_out.insert(0, 'text_id', f_id)
    
    res = pd.merge(f_in, f_out, how='outer', on=['text_id', 'entity_id'])
    res = pd.merge(res, ref_obt, how='left', on='dict_id')
    res = pd.merge(res, ref_tax, how='left', on='dict_id')
    res['dict_name'] = res.dict_name_x.fillna(res.dict_name_y)
    res.drop(columns=['dict_name_x', 'dict_name_y'], inplace=True)
    # print(res.head())
    # break

/Users/zora/anaconda/envs/bacbiotope/lib/python3.6/site-packages/ipykernel_launcher.py:14: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


    text_id entity_id                                               name  \
0  15358511        T1  Attachment of Escherichia coli O157:H7 grown i...   
1  15358511        T2  This study investigated the effect of growth i...   
2  15358511        T3                                         Attachment   
3  15358511        T4                           Escherichia coli O157:H7   
4  15358511        T5                                  tryptic soy broth   

        category positions standard_id      dict_type     dict_id  \
0          Title     0 189         NaN            NaN         NaN   
1      Paragraph  190 1044         NaN            NaN         NaN   
2      Phenotype      0 10          N1    OntoBiotope  OBT:000029   
3  Microorganism     14 38          N2  NCBI_Taxonomy       83334   
4        Habitat     48 65          N3    OntoBiotope  OBT:000360   

     dict_name_class                   dict_name  
0                NaN                         NaN  
1                NaN      

In [90]:
pattern='.*norm-(.*)\..*'
header=True
for f in input_file:
    f_id = re.search(pattern, f).group(1)
    
    # Parse input files, with format of: label \t category \s [start \s end]rep  \t name
    f_in = pd.read_csv(os.path.join(file_path, f), sep='\t', header=None)
    f_in.columns=['entity_id', 'mix', 'name']
    f_in[['category', 'positions']] = f_in.mix.str.split(n=1, expand=True)
    f_in.drop(columns=['mix'], inplace=True)
    f_in.insert(0, 'text_id', f_id)

    # Parse output files, with format of: label \t category \s entity_lable \s referent
    f_out = pd.read_csv(os.path.join(file_path, f[:-1]+'2'), sep='\t|\s', header=None)
    f_out.columns=['standard_id', 'dict_type', 'entity_id', 'dict_id']
    f_out.entity_id = f_out.entity_id.str.lstrip('Annotation:')
    f_out.dict_id = f_out.dict_id.str.lstrip('Referent:')    
    f_out.insert(0, 'text_id', f_id)
    
    res = pd.merge(f_in, f_out, how='outer', on=['text_id', 'entity_id'])
    res = pd.merge(res, ref_obt, how='left', on='dict_id')
    res = pd.merge(res, ref_tax, how='left', on='dict_id', suffixes=('', '_tmp'))
    res['dict_name'] = res.dict_name.fillna(res.dict_name_tmp)
    res.drop(columns=['dict_name_tmp'], inplace=True)
    print(res.head())
    break

/Users/zora/anaconda/envs/bacbiotope/lib/python3.6/site-packages/ipykernel_launcher.py:14: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


    text_id entity_id                                               name  \
0  15358511        T1  Attachment of Escherichia coli O157:H7 grown i...   
1  15358511        T2  This study investigated the effect of growth i...   
2  15358511        T3                                         Attachment   
3  15358511        T4                           Escherichia coli O157:H7   
4  15358511        T5                                  tryptic soy broth   

        category positions standard_id      dict_type     dict_id  \
0          Title     0 189         NaN            NaN         NaN   
1      Paragraph  190 1044         NaN            NaN         NaN   
2      Phenotype      0 10          N1    OntoBiotope  OBT:000029   
3  Microorganism     14 38          N2  NCBI_Taxonomy       83334   
4        Habitat     48 65          N3    OntoBiotope  OBT:000360   

                    dict_name    dict_name_class  
0                         NaN                NaN  
1                         